# Labelme转COCO-单个文件

同济子豪兄 呕心沥血写成

兼容目标检测、图像分割、关键点检测三种标注

2023-3-9

2023-3-10

2023-4-15

2023-4-16

## 导入工具包

In [1]:
import os
import json
import numpy as np

## 删除系统自动生成的多余文件

建议在 Linux 系统中运行爬虫、划分训练集测试集代码

### 查看待删除的多余文件

In [2]:
!find . -iname '__MACOSX'

In [3]:
!find . -iname '.DS_Store'

In [4]:
!find . -iname '.ipynb_checkpoints'

./.ipynb_checkpoints


### 删除多余文件

In [5]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

In [6]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

In [7]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

### 验证多余文件已删除

In [8]:
!find . -iname '__MACOSX'

In [9]:
!find . -iname '.DS_Store'

In [10]:
!find . -iname '.ipynb_checkpoints'

## 载入一个labelme格式的json标注文件

In [11]:
with open('DSC_0281.json', 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [12]:
labelme.keys()

dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth'])

## 元数据 

In [13]:
labelme['version']

'5.1.1'

In [14]:
labelme['flags']

{}

In [15]:
# 图像文件名
labelme['imagePath']

'DSC_0281.jpg'

In [16]:
labelme['imageData']

In [17]:
# 图像高度
labelme['imageHeight']

3712

In [18]:
# 图像宽度
labelme['imageWidth']

5568

## 标注信息

In [19]:
labelme['shapes']

[{'label': 'sjb_rect',
  'points': [[1138.0, 0.0], [4480.0, 1562.0]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'sjb_rect',
  'points': [[1160.0, 1954.0], [4560.0, 3566.0]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'angle_30',
  'points': [[4434.0, 284.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_30',
  'points': [[4526.0, 2316.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_60',
  'points': [[1202.0, 44.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_60',
  'points': [[1218.0, 2000.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_90',
  'points': [[1934.0, 1512.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_90',
  'points': [[1906.0, 3532.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'sjb_poly',
  'points':

## 创建coco格式的字典

In [20]:
coco = {}

## info

In [21]:
# coco['info'] = {}
# coco['info']['description'] = 'Labelme2coco keypoint format script from Zihao'
# coco['info']['year'] = 2023
# coco['info']['date_created'] = '2023/03/09'

## categories

supercategory：框的类别（对于人体姿态估计，只有person一个类别）

keypoints：节点类别排序，后续的关键点坐标展示顺序

skeleton：可有可无，可视化时需要

In [22]:
class_list= {
    'supercategory': 'sjb_rect',
    'id': 1,
    'name': 'sjb_rect',
    'keypoints': ['angle_30', 'angle_60', 'angle_90'], # 大小写敏感
    'skeleton':[[0,1], [0,2], [1,2]]
}

In [23]:
coco['categories'] = []
coco['categories'].append(class_list)

coco['images'] = []
coco['annotations'] = []

IMG_ID = 0
ANN_ID = 0

## 函数-处理单个labelme标注json文件

In [24]:
def process_single_json(labelme, image_id=1):
    '''
    输入labelme的json数据，输出coco格式的每个框的关键点标注信息
    '''
    
    global ANN_ID
    
    coco_annotations = []
    
    for each_ann in labelme['shapes']: # 遍历该json文件中的所有标注

        if each_ann['shape_type'] == 'rectangle': # 筛选出框

            # 该框元数据
            bbox_dict = {}
            bbox_dict['category_id'] = 1
            bbox_dict['segmentation'] = []
            
            bbox_dict['iscrowd'] = 0
            bbox_dict['segmentation'] = []
            bbox_dict['image_id'] = image_id
            bbox_dict['id'] = ANN_ID
            # print(ANN_ID)
            ANN_ID += 1

            # 获取该框坐标
            bbox_left_top_x = min(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_left_top_y = min(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_right_bottom_x = max(int(each_ann['points'][0][0]), int(each_ann['points'][1][0]))
            bbox_right_bottom_y = max(int(each_ann['points'][0][1]), int(each_ann['points'][1][1]))
            bbox_w = bbox_right_bottom_x - bbox_left_top_x
            bbox_h = bbox_right_bottom_y - bbox_left_top_y
            bbox_dict['bbox'] = [bbox_left_top_x, bbox_left_top_y, bbox_w, bbox_h] # 左上角x、y、框的w、h
            bbox_dict['area'] = bbox_w * bbox_h
            
            # 筛选出分割多段线
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'polygon': # 筛选出分割多段线标注
                    # 第一个点的坐标
                    first_x = each_ann['points'][0][0]
                    first_y = each_ann['points'][0][1]
                    if (first_x>bbox_left_top_x) & (first_x<bbox_right_bottom_x) & (first_y<bbox_right_bottom_y) & (first_y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_dict['segmentation'] = list(map(lambda x: list(map(lambda y: round(y, 2), x)), each_ann['points'])) # 坐标保留两位小数
                        # bbox_dict['segmentation'] = each_ann['points']

            # 筛选出该个体框中的所有关键点
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: # 遍历所有标注
                
                if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                    # 关键点横纵坐标
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_left_top_x) & (x<bbox_right_bottom_x) & (y<bbox_right_bottom_y) & (y>bbox_left_top_y): # 筛选出在该个体框中的关键点
                        bbox_keypoints_dict[label] = [x, y]
                        
            bbox_dict['num_keypoints'] = len(bbox_keypoints_dict)
            # print(bbox_keypoints_dict)

            # 把关键点按照类别顺序排好
            bbox_dict['keypoints'] = []
            for each_class in class_list['keypoints']:
                if each_class in bbox_keypoints_dict:
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][0])
                    bbox_dict['keypoints'].append(bbox_keypoints_dict[each_class][1])
                    bbox_dict['keypoints'].append(2) # 2-可见不遮挡 1-遮挡 0-没有点
                else: # 不存在的点，一律为0
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    bbox_dict['keypoints'].append(0)
                    
            coco_annotations.append(bbox_dict)
            
    return coco_annotations

## 测试一下函数的效果，处理单个labelme格式的json标注文件

In [25]:
labelme_json_path = 'DSC_0281.json'

with open(labelme_json_path, 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [26]:
process_single_json(labelme)

[{'category_id': 1,
  'segmentation': [[1202.0, 28.0], [4454.0, 280.0], [1938.0, 1524.0]],
  'iscrowd': 0,
  'image_id': 1,
  'id': 0,
  'bbox': [1138, 0, 3342, 1562],
  'area': 5220204,
  'num_keypoints': 3,
  'keypoints': [4434, 284, 2, 1202, 44, 2, 1934, 1512, 2]},
 {'category_id': 1,
  'segmentation': [[1222.0, 1992.0], [4542.0, 2292.0], [1918.0, 3532.0]],
  'iscrowd': 0,
  'image_id': 1,
  'id': 1,
  'bbox': [1160, 1954, 3400, 1612],
  'area': 5480800,
  'num_keypoints': 3,
  'keypoints': [4526, 2316, 2, 1218, 2000, 2, 1906, 3532, 2]}]

## images和annotations

In [27]:
IMG_ID = 0
ANN_ID = 0

In [28]:
# 遍历所有 labelme 格式的 json 文件
for labelme_json in os.listdir(): 
    
    if labelme_json.split('.')[-1] == 'json':
        
        with open(labelme_json, 'r', encoding='utf-8') as f:
            
            labelme = json.load(f)
            
            ## 提取图像元数据
            img_dict = {}
            img_dict['file_name'] = labelme['imagePath']
            img_dict['height'] = labelme['imageHeight']
            img_dict['width'] = labelme['imageWidth']
            img_dict['id'] = IMG_ID
            coco['images'].append(img_dict)
            
            ## 提取框和关键点信息
            coco_annotations = process_single_json(labelme, image_id=IMG_ID)
            coco['annotations'] += coco_annotations
            
            IMG_ID += 1
            
            print(labelme_json, '已处理完毕')

    else:
        pass

DSC_0281.json 已处理完毕


## 保存为MS COCO格式的json标注文件

In [29]:
if not os.path.exists('output_coco'):
    os.mkdir('output_coco')
    print('创建新目录 output_coco')

In [30]:
coco_path = 'output_coco/coco_sample.json'

In [31]:
with open(coco_path, 'w') as f:
    json.dump(coco, f, indent=2)

In [32]:
coco

{'categories': [{'supercategory': 'sjb_rect',
   'id': 1,
   'name': 'sjb_rect',
   'keypoints': ['angle_30', 'angle_60', 'angle_90'],
   'skeleton': [[0, 1], [0, 2], [1, 2]]}],
 'images': [{'file_name': 'DSC_0281.jpg',
   'height': 3712,
   'width': 5568,
   'id': 0}],
 'annotations': [{'category_id': 1,
   'segmentation': [[1202.0, 28.0], [4454.0, 280.0], [1938.0, 1524.0]],
   'iscrowd': 0,
   'image_id': 0,
   'id': 0,
   'bbox': [1138, 0, 3342, 1562],
   'area': 5220204,
   'num_keypoints': 3,
   'keypoints': [4434, 284, 2, 1202, 44, 2, 1934, 1512, 2]},
  {'category_id': 1,
   'segmentation': [[1222.0, 1992.0], [4542.0, 2292.0], [1918.0, 3532.0]],
   'iscrowd': 0,
   'image_id': 0,
   'id': 1,
   'bbox': [1160, 1954, 3400, 1612],
   'area': 5480800,
   'num_keypoints': 3,
   'keypoints': [4526, 2316, 2, 1218, 2000, 2, 1906, 3532, 2]}]}

## 验证MS COCO格式的标注

In [ ]:
# !pip install pycocotools

In [33]:
from pycocotools.coco import COCO

my_coco = COCO(coco_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
